# Pinecone Canopy library quick start notebook

**Canopy** is a Sofware Development Kit (SDK) for AI applications. Canopy allows you to test, build and package Retrieval Augmented Applications with Pinecone Vector Database. 

This notebook introduce the quick start steps for working with Canopy library. You can find more details about this project and advanced use in the project [documentaion](../README.md).


## Prerequisites

install canopy library

In [ ]:
!pip install -qU pinecone-canopy

By default, Canopy uses Pinecone and OpenAI so we need to configure the related API keys.

To get Pinecone free trial API key and environment register or log into your Pinecone account in the [console](https://app.pinecone.io/). You can access your API key from the "API Keys" section in the sidebar of your dashboard, and find the environment name next to it.

You can find your free trial OpenAI API key [here](https://platform.openai.com/account/api-keys). You might need to login or register to OpenAI services.



In [ ]:
import os

os.environ["PINECONE_API_KEY"] = os.environ.get('PINECONE_API_KEY') or 'YOUR_PINECONE_API_KEY'
os.environ["PINECONE_ENVIRONMENT"] = os.environ.get('PINECONE_ENVIRONMENT') or 'PINECONE_ENVIRONMENT'
os.environ["OPENAI_API_KEY"] = os.environ.get('OPENAI_API_KEY') or 'OPENAI_API_KEY'

We don't have to do the following step since openai loads the environment variable on import.

When working with Jupyter notebook we'll have to restart the kernel for any mistake in this variable so it's safer to explicitly set the api key.

In [8]:
import openai

openai.api_key = os.environ["OPENAI_API_KEY"]

## Pinecone Documentation Dataset

Now we'll load a crawl of from 25/10/23 of pinecone docs [website](https://docs.pinecone.io/docs/).

We will use this data to demonstrate how to build a RAG pipepline to answer questions about Pinecone DB.

In [ ]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

data = pd.read_parquet("https://storage.googleapis.com/pinecone-datasets-dev/pinecone_docs_ada-002/raw/file1.parquet")
data.head()

Each record in this dataset represents a single page in Pinecone's documentation. Each row contatins a unique id, the raw text of the page in markdown language, the url of the page as "source" and some metadata. 

## Init a Tokenizer


Many of Canopy's components are using tokenization, which is a process that splits text into tokens - basic units of text (like word or sub-words) that are used for processing. Therefore, Canopy uses a singleton `Tokenizer` object which needs to be initialized once.

In [ ]:
from canopy.tokenizer import Tokenizer
Tokenizer.initialize()

After initilizing the global object, we can simply create an instance from anywhere in our code, without providing any parameters:

In [ ]:
from canopy.tokenizer import Tokenizer

tokenizer = Tokenizer()

tokenizer.tokenize("Hello world!")

## Creating a KnowledgBase to store our data for search

The `KnowledgeBase` object is responsible for storing and indexing textual documents.

Once documents were indexed, the `KnowledgeBase` can be queried with a new unseen text passage, for which the most relevant document chunks are retrieved.

The `KnowledgeBase` holds a connection to a Pinecone index and provides a simple API to insert, delete and search textual documents.

The `KnoweldgeBase`'s `upsert()` operation is used to index new documents, or update already stored documents. The `upsert` process splits each document's text into smaller chunks, transforms these chunks to vector embeddings, then upserts those vectors to the underlying Pinecone index. At Query time, the `KnowledgeBase` transforms the textual query text to a vector in a similar manner, then queries the underlying Pinecone index to retrieve the top-k most closely matched document chunks.

Here we create a `KnowledgeBase` with our desired index name: 

In [ ]:
from canopy.knowledge_base import KnowledgeBase

INDEX_NAME = "my-index"

kb = KnowledgeBase(index_name=INDEX_NAME)

In the first one-time setup of a new Canopy service, an underlying Pinecone index needs to be created. If you have created a Canopy-enabled Pinecone index before - you can skip this step.

Note: Since Canopy uses a dedicated data schema, it is not recommended to use a pre-existing Pinecone index that wasn't created by Canopy's `create_canopy_index()` method.

In [ ]:
from canopy.knowledge_base import list_canopy_indexes

if not any(name.endswith(INDEX_NAME) for name in list_canopy_indexes()):
    kb.create_canopy_index(indexed_fields=["title"])

You can see the index created in Pinecone's [console](https://app.pinecone.io/)

next time we would like to init a knowledge base instance to this index, we can simply call the connect method:

In [ ]:
kb = KnowledgeBase(index_name=INDEX_NAME)
kb.connect()

> 💡 Note: a knowledge base must be connected to an index before excuting any operation. You should call `kb.connect()` to connect  an existing index or call `kb.create_canopy_index(INDEX_NANE)` before calling any other method of the KB 

## Upsert data to our KnowledgBase

First, we need to convert our dataset to list of `Document` objects

Each document object can hold id, text, source and metadata:

In [ ]:
from canopy.models.data_models import Document

example_docs = [Document(id="1",
                      text="This is text for example",
                      source="https://url.com"),
                Document(id="2",
                        text="this is another text",
                        source="https://another-url.com",
                        metadata={"my-key": "my-value"})]

Luckily the columns in our dataset fits this scehma, so we can use a simple iteration to prepare our data:

In [ ]:
documents = [Document(**row) for _, row in data.iterrows()]

Now we are ready to upsert our data, with only a single command:

In [ ]:
from tqdm.auto import tqdm

batch_size = 10

for i in tqdm(range(0, len(documents), batch_size)):
    kb.upsert(documents[i: i+batch_size])

Internally, the KnowledgeBase handle for use all the processing needed to load data into Pinecone. It chunks the text to smaller pieces and encode them to vectors (embeddings) that can be then upserted directly to Pinecone. Later in this notebook we'll learn how to tune and costumize this process.

## Query the KnowledgeBase

Now we can query the knowledge base. The KnowledgeBase will use its default parameters like `top_k` to exectute the query:

In [ ]:
def print_query_results(results):
    for query_results in results:
        print('query: ' + query_results.query + '\n')
        for document in query_results.documents:
            print('document: ' + document.text.replace("\n", "\\n"))
            print('source: ' + document.source)
            print(f"score: {document.score}\n")

In [ ]:
from canopy.models.data_models import Query
results = kb.query([Query(text="p1 pod capacity")])

print_query_results(results)

We can also use metadata filtering and specify `top_k`:

In [ ]:
from canopy.models.data_models import Query
results = kb.query([Query(text="p1 pod capacity",
                          metadata_filter={"title": "limits"},
                          top_k=2)])

print_query_results(results)

As you can see above, using the metadata filter we get results only from the "limits" page

## Query the Context Engine

`ContextEngine` is an object that responsible to retrieve the most relevant context for a given query and token budget.  

While `KnowledgeBase` retreivs the full `top-k` structred documens for each query including all the metadata related to them, context engine in charge of transforming this information to a "prompt ready" context that can later feeded to an LLM. To achieve this the context engine holds a `ContextBuilder` object that takes query results from the knowledge base and returns a `Context` object. The context builder also considers the `max_context_tokens` budget given to it and build the most relevant context that not exceeds the token budget.

In [ ]:
from canopy.context_engine import ContextEngine
context_engine = ContextEngine(kb)

In [ ]:
import json

result = context_engine.query([Query(text="capacity of p1 pods", top_k=5)], max_context_tokens=512)

print(result.to_text(indent=2))
print(f"\n# tokens in context returned: {result.num_tokens}")

As you can see above, although we set `top_k=5`, context engine retreived only 3 results in order to satisfy the 512 tokens limit. Also, the documents in the context contain only the text and source and not all the metadata that is not necessarily needed by the LLM. 

## Knowledgeable chat engine

Now we are ready to start chatting with our data!

Canopy's `ChatEngine` is a one-stop-shop RAG-infused Chatbot. The `ChatEngine` wraps an underlying LLM such as OpenAI's ChatGPT, enhancing it by providing relevant context from the user's knowledge base. It also automatically phrases search queries out of the chat history and send them to the knowledge base.

In [ ]:
from canopy.chat_engine import ChatEngine
chat_engine = ChatEngine(context_engine)

In [ ]:
from typing import Tuple
from canopy.models.data_models import Messages, UserMessage, AssistantMessage

def chat(new_message: str, history: Messages) -> Tuple[str, Messages]:
    messages = history + [UserMessage(content=new_message)]
    response = chat_engine.chat(messages)
    assistant_response = response.choices[0].message.content
    return assistant_response, messages + [AssistantMessage(content=assistant_response)]

In [ ]:
from IPython.display import display, Markdown

history = []
response, history = chat("What is the capacity of p1 pods?", history)
display(Markdown(response))

In [ ]:
response, history = chat("And for what latency requirements does it fit?", history)
display(Markdown(response))

> 💡 Note: Canopy calls the underlying LLM, providing both the user-provided chat history and a generated `Context` prompt. This might surpass the `ChatEngine`'s configured `max_prompt_tokens`. By default, the `ChatEngine` would truncate the older most messages in the chat history avoid exceeding this limit. This behavior in configurable, as explained in the [documentation](https://github.com/pinecone-io/canopy/blob/main/src/canopy/chat_engine/chat_engine.py)

## Costumization Example

Canopy built as a modular library, where each component can fully be costumized by the user.

Before we start, we would like to have a quick overview of the inner components used by the knowledge base:

- **Index**: A Pinecone index that holds the vector representations of the documents.
- **Chunker**: A `Chunker` object that is used to chunk the documents into smaller pieces of text.
- **Encoder**: An `RecordEncoder` object that is used to encode the chunks and queries into vector representations.

In the following example, we show how you can costumize the `Chunker` component used by the knowledge base.

First, we will create a dummy chunker class that simply chunks the text by new lines `\n`.

In [ ]:
from typing import List
from canopy.knowledge_base.chunker.base import Chunker
from canopy.knowledge_base.models import KBDocChunk

class NewLineChunker(Chunker):

     def chunk_single_document(self, document: Document) -> List[KBDocChunk]:
        line_chunks = [chunk
                       for chunk in document.text.split("\n")]
        return [KBDocChunk(id=f"{document.id}_{i}",
                           document_id=document.id,
                           text=text_chunk,
                           source=document.source,
                           metadata=document.metadata)
                for i, text_chunk in enumerate(line_chunks)]
    
     async def achunk_single_document(self, document: Document) -> List[KBDocChunk]:
        raise NotImplementedError()

In [ ]:
chunker = NewLineChunker()

document = Document(id="id1",
                    text="This is first line\nThis is the second line",
                    source="example",
                    metadata={"title": "newline"})
chunker.chunk_single_document(document)

Now we can initialize a new knowledge base to use our new chunker:

In [ ]:
kb = KnowledgeBase(index_name=INDEX_NAME,
                   chunker=chunker)
kb.connect()

And upsert our example document:

In [ ]:
kb.upsert([document])

In [ ]:
results = kb.query([Query(text="second line",
                          metadata_filter={"title": "newline"})])

print_query_results(results)

As we can see above, our knowledge base split the document by new line as expected.

Delete the index once you are sure that you do not want to use it anymore. Once the index is deleted, you cannot use it again.

In [ ]:
kb.delete_index()